In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm
from matplotlib.gridspec import GridSpec

# Import colormaps and limits
import sys
sys.path.append('./')
from colormaps import *

In [ ]:
data_dir = '../../Zenodo/model_output'

spinup = xr.open_dataset(f'{data_dir}/spinup.nc')

# QU_M, PI_M, PL_M
M_QUN = xr.open_dataset(f'{data_dir}/QU_M.nc')
M_PIN = xr.open_dataset(f'{data_dir}/PI_M.nc')
M_PLN = xr.open_dataset(f'{data_dir}/PL_M.nc')

# QU_H, PI_H, PL_H
H_QUN = xr.open_dataset(f'{data_dir}/QU_H.nc')
H_PIN = xr.open_dataset(f'{data_dir}/PI_H.nc')
H_PLN = xr.open_dataset(f'{data_dir}/PL_H.nc')

# QU_H, PI_H, PL_H
H_QUN_retuned = xr.open_dataset(f'{data_dir}/retuned/QU_H_retuned.nc')
H_PIN_retuned = xr.open_dataset(f'{data_dir}/retuned/PI_H_retuned.nc')
H_PLN_retuned = xr.open_dataset(f'{data_dir}/retuned/PL_H_retuned.nc')

# QU_M, PI_M, PL_M
M_QUN_calving = xr.open_dataset(f'{data_dir}/calving/QU_M_calving.nc')
M_PIN_calving = xr.open_dataset(f'{data_dir}/calving/PI_M_calving.nc')
M_PLN_calving = xr.open_dataset(f'{data_dir}/calving/PL_M_calving.nc')

# QU_H, PI_H, PL_H
H_QUN_calving = xr.open_dataset(f'{data_dir}/calving/QU_H_calving.nc')
H_PIN_calving = xr.open_dataset(f'{data_dir}/calving/PI_H_calving.nc')
H_PLN_calving = xr.open_dataset(f'{data_dir}/calving/PL_H_calving.nc')

In [ ]:
def animate_laddie_melt_draft_icevel(dsIMAU, tslice, xslice, fps, mask=[], savename=None):
    
    """ 
    dsIMAU      :  file IMAU-ICE output help_fields_ANT.nc
    tslice      :  array of time slices to show, e.g. np.arange(0,1001,10)
    xslice      :  x domain boundaries, e.g. slice(-200000,250000) 
    fps         :  frames per second 
    savename    :  directory and filename to save the animation
    """
        
    fig,ax = plt.subplots(1,3, figsize=(9,9
                                    ),dpi=200)
    
    snapshots_melt, snapshots_draft, snapshots_ice = [],[],[]
    quivUice, quivVice = [],[]
    mask_gr, mask_oc, mask_GL = [],[],[]
    cont_draft = [[]]
    BMB_vals = []


    for t in tslice:

        # Select time slice and x slice
        dsIM = dsIMAU.sel(time=t, x = xslice)

        # Transpose data
        newx = dsIM.y[::-1]
        newy = dsIM.x
        newmelt = -1*dsIM.BMB.T
        newmask = dsIM.mask.T
        newzb = dsIM.Hs.T-dsIM.Hi.T
        newicevelu = -dsIM['v_surf'].T
        newicevelv = dsIM['u_surf'].T
        newmaskgl = xr.where(dsIM['mask']==7, 1,0).T

        if t == 0:
            # Select time slice and x slice
            dsIM = spinup.sel(time=50000, x = xslice)
            newicevelu = -dsIM['v_surf'].T
            newicevelv = dsIM['u_surf'].T
            newmaskgl = xr.where(dsIM['mask']==7, 1,0).T

        snapshots_melt.append(newmelt)
        snapshots_draft.append(newzb)
        snapshots_ice.append(np.sqrt(newicevelu**2+newicevelv**2))

        BMB_vals.append(np.nansum(newmelt*2000*2000*918*1e-12))

        quiver_slice = (slice(None, None, 10), slice(None, None, 10))  # Slicing every other cell
        quivxice = newx[quiver_slice[1]]/1000
        quivyice = newy[quiver_slice[0]]/1000
        
        quivUice.append(newicevelu[quiver_slice])
        quivVice.append(newicevelv[quiver_slice])

        mask_oc.append(xr.where(newmask==1, 1, np.nan))
        mask_gr.append(xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan))
        mask_GL.append(xr.where(newmaskgl==1, 1, np.nan))


    # Plot draft
    im_draft = ax[0].pcolormesh( newx/1000, newy/1000, snapshots_draft[0], cmap='cmo.deep_r', norm = mpl.colors.Normalize(vmin=-700, vmax=0))
    cont_draft[0] = ax[0].contour(newx/1000, newy/1000, snapshots_draft[0],  [-300, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=1)
    h, l = cont_draft[0].legend_elements()
    cbar = plt.colorbar(im_draft, ax = ax[0],  orientation='horizontal', label= 'Ice draft depth [m]', pad = 0.05)

    ax[0].set_ylabel('[km]')
    ax[0].set_yticks([-150, -100,-50, 0, 50, 100, 150, 200, 250])
    ax[0].legend(h,['-300 m', '-250 m', '-225 m', '-200 m', '-175 m', '-150 m', '-125 m', '-100 m'],bbox_to_anchor=(-1.0, 1),loc=2)
   
    # Plot melt
    im_melt = ax[1].pcolormesh( newx/1000, newy/1000, snapshots_melt[0], cmap=cmap_BMB, norm = norm_BMB)
    cbar = plt.colorbar(im_melt, ax = ax[1],  ticks=[-10,-1, 0, 1, 10, 100], orientation='horizontal', label= 'Sub-shelf melt [m/yr]', pad = 0.05)
    cbar.ax.set_xticklabels([-10,-1, 0, 1, 10, 100]);
    bmbval = ax[1].text(-38,-195,f'Melt flux: {BMB_vals[0]:.2f} Gt/yr',zorder=7)

    # Plot ice vel
    im_ice = ax[2].pcolormesh( newx/1000, newy/1000, snapshots_ice[0], cmap=cmap_u_surf, norm = norm_u_surf)
    plt.colorbar(im_ice, ax = ax[2], orientation='horizontal', label= 'Ice surface velocity [m/s]', pad = 0.05)
    Qice = ax[2].quiver(quivxice, quivyice, quivUice[0], quivVice[0], scale=2000, width=1/100, facecolor='white', )

    im_oc, im_gr = [[],[],[]],[[],[],[]]

    for j, axs in enumerate([ax[0], ax[1]]):
        im_oc[j] = axs.pcolormesh(newx/1000, newy/1000,  mask_oc[0],      cmap=cmap_ocean,    norm=norm_ocean, zorder=6)
        im_gr[j] = axs.pcolormesh(newx/1000, newy/1000,  mask_gr[0],      cmap=cmap_ground,   norm=norm_ground, zorder=6)
    
    im_oc[2] = ax[2].pcolormesh(newx/1000, newy/1000,  mask_oc[0],      cmap=cmap_ocean,    norm=norm_ocean, zorder=9)
    im_gl = ax[2].pcolormesh(newx/1000, newy/1000,  mask_GL[0],      cmap=cmap_GL,   norm=norm_GL, zorder=6)

    plt.suptitle(f'Model year {tslice[0]}', weight='bold', size=15);

    fig.tight_layout()

  
    
    def animate_func(i):
        if i % fps == 0:
            print( '.', end = '')

        # Clear the previous contours before plotting new ones
        for coll in cont_draft[0].collections:
            coll.remove()
        cont_draft[0] = ax[0].contour(newx/1000, newy/1000, snapshots_draft[i], [-300, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=5)

        for j in range(2):
            im_oc[j].set_array(mask_oc[i])
            im_gr[j].set_array(mask_gr[i])
        
        im_oc[2].set_array(mask_oc[i])

        im_gl.set_array(mask_GL[i])
        
        im_melt.set_array(snapshots_melt[i]) 
        im_draft.set_array(snapshots_draft[i]) 
        im_ice.set_array(snapshots_ice[i]) 

        bmbval.set_text(f'Melt flux: {BMB_vals[i]:.2f} Gt/yr')

        Qice.set_UVC(quivUice[i],quivVice[i])

        fig.suptitle(f'Model year {tslice[i]}', weight='bold', size=20);

        return cont_draft[0].collections


    anim = animation.FuncAnimation(
                               fig, 
                               animate_func, 
                               frames = len(tslice*10),
                               interval = 1000 / 400, blit=False# in ms
                               )

    
    #if savename is not None:
    anim.save(f'{savename}.mp4', fps=fps, extra_args=['-vcodec', 'libx264']) 

    return 

In [ ]:
animate_laddie_melt_draft_icevel(M_QUN, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='QU_M_animation_fields')
animate_laddie_melt_draft_icevel(M_PIN, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PI_M_animation_fields')
animate_laddie_melt_draft_icevel(M_PLN, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PL_M_animation_fields')

In [ ]:
animate_laddie_melt_draft_icevel(H_QUN, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='QU_H_animation_fields')
animate_laddie_melt_draft_icevel(H_PIN, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PI_H_animation_fields')
animate_laddie_melt_draft_icevel(H_PLN, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PL_H_animation_fields')


In [ ]:
animate_laddie_melt_draft_icevel(H_QUN_retuned, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='QU_H_retuned_animation_fields')
animate_laddie_melt_draft_icevel(H_PIN_retuned, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PI_H_retuned_animation_fields')
animate_laddie_melt_draft_icevel(H_PLN_retuned, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PL_H_retuned_animation_fields')

In [ ]:
animate_laddie_melt_draft_icevel(M_QUN_calving, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='QU_M_calving_animation_fields')
animate_laddie_melt_draft_icevel(M_PIN_calving, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PI_M_calving_animation_fields')
animate_laddie_melt_draft_icevel(M_PLN_calving, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PL_M_calving_animation_fields')

In [ ]:
animate_laddie_melt_draft_icevel(H_QUN_calving, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='QU_H_calving_animation_fields')
animate_laddie_melt_draft_icevel(H_PIN_calving, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PI_H_calving_animation_fields')
animate_laddie_melt_draft_icevel(H_PLN_calving, np.arange(0,1001,10), slice(-200000,250000), fps=5, mask=[], savename='PL_H_calving_animation_fields')